# Benchmark of sympy's partial_velocity() function

In this notebook, we explore and benchmark a new method for calculating partial velocities in a multibody dynamic system. The aim is to leverage the linearity relationship between velocities and generalized speeds to optimize computations.

## Motivation

Traditionally, calculating partial velocities in the context of multibody dynamics involved the use of differentiation (`diff()`) and Jacobian (`jacobian()`) functions to derive linear coefficients. However, this approach has been identified as inefficient in certain scenarios, particularly within the mechanics module of SymPy.

## Issue Reference

The inspiration for this new method arises from Issue [#25070 on SymPy's GitHub repository](https://github.com/sympy/sympy/issues/25070), where the inefficiencies of the current approach were discussed in detail. The issue highlighted the computational overhead associated with the existing methods when deriving linear coefficients assumed to be linear.

## Implementation in This Notebook

The following analyses are performed:

- Comparison of the old and new methods for calculating partial velocities.
- Execution of two different benchmarks to test and compare performances.

To ensure accurate results, it is advised to perform a "Restart & Run All" within this Jupyter notebook. This step guarantees that cells are executed in the correct order, especially as this notebook tests functionalities that must be executed with specific sequence constraints.


# Setup

In [11]:
import sympy as sm
import sympy.physics.mechanics as me
from sympy.physics.mechanics.models import n_link_pendulum_on_cart
from sympy.physics.mechanics.functions import _f_list_parser
from sympy.utilities.iterables import iterable
from sympy.physics.mechanics.particle import Particle
from sympy.physics.mechanics.rigidbody import RigidBody

# Generate the system with one pendulum link on a cart
sys_kane = n_link_pendulum_on_cart(n=30, cart_force=True, joint_torques=False)

v = sm.symbols('v')

frame = sys_kane._inertial

# Extract coordinates (generalized coordinates)
coordinates = sys_kane.q

# Extract generalized speeds
speeds = [us for us in sys_kane.u]

# Extract velocities of all particles
# In n_link_pendulum_on_cart, particles are the masses of the pendulum bobs
velocities = [body.masscenter.vel(frame) for body in sys_kane.bodies]
velocities[2] += v*5*frame.x + speeds[2]*3 *frame.y

In [12]:
from sympy import sin, cos
from sympy.physics.mechanics.functions import (msubs, find_dynamicsymbols,
                                               _f_list_parser,
                                               _validate_coordinates,
                                               _parse_linear_solver)

def test_issue_24887():
    # Spherical pendulum
    g, l, m, c = sm.symbols('g l m c')
    q1, q2, q3, u1, u2, u3 = me.dynamicsymbols('q1:4 u1:4')
    N = me.ReferenceFrame('N')
    A = me.ReferenceFrame('A')
    A.orient_body_fixed(N, (q1, q2, q3), 'zxy')
    N_w_A = A.ang_vel_in(N)
    # A.set_ang_vel(N, u1 * A.x + u2 * A.y + u3 * A.z)
    kdes = [N_w_A.dot(A.x) - u1, N_w_A.dot(A.y) - u2, N_w_A.dot(A.z) - u3]
    O = me.Point('O')
    O.set_vel(N, 0)
    Po = O.locatenew('Po', -l * A.y)
    Po.set_vel(A, 0)
    P = Particle('P', Po, m)
    
    kane = me.KanesMethod(N, [q1, q2, q3], [u1, u2, u3], kdes, bodies=[P],
                       forcelist=[(Po, -m * g * N.y)])
    
    kane.kanes_equations()
    
    expected_md = m * l ** 2 * sm.Matrix([[1, 0, 0], [0, 0, 0], [0, 0, 1]])
    
    expected_fd = sm.Matrix([
        [l*m*(g*(sin(q1)*sin(q3) - sin(q2)*cos(q1)*cos(q3)) - l*u2*u3)],
        [0], [l*m*(-g*(sin(q1)*cos(q3) + sin(q2)*sin(q3)*cos(q1)) + l*u1*u2)]])
    
    assert me.find_dynamicsymbols(kane.forcing).issubset({q1, q2, q3, u1, u2, u3})
    assert sm.simplify(kane.mass_matrix - expected_md) == sm.zeros(3, 3)
    assert sm.simplify(kane.forcing - expected_fd) == sm.zeros(3, 1)

    frame = kane._inertial
    vel_list = [body.masscenter.vel(frame) for body in kane.bodies]
    vel_vecs = [msubs(i, kane._qdot_u_map) for i in vel_list]
    map = kane._qdot_u_map
    gen_speeds = [us for us in kane.u]

    return frame, vel_vecs, gen_speeds, map 

In [13]:
def partial_velocity(vel_vecs, gen_speeds, frame):
    """Returns a list of partial velocities with respect to the provided
    generalized speeds in the given reference frame for each of the supplied
    velocity vectors.

    The output is a list of lists. The outer list has a number of elements
    equal to the number of supplied velocity vectors. The inner lists are, for
    each velocity vector, the partial derivatives of that velocity vector with
    respect to the generalized speeds supplied.

    Parameters
    ==========

    vel_vecs : iterable
        An iterable of velocity vectors (angular or linear).
    gen_speeds : iterable
        An iterable of generalized speeds.
    frame : ReferenceFrame
        The reference frame that the partial derivatives are going to be taken
        in.

    Examples
    ========

    >>> from sympy.physics.vector import Point, ReferenceFrame
    >>> from sympy.physics.vector import dynamicsymbols
    >>> from sympy.physics.vector import partial_velocity
    >>> u = dynamicsymbols('u')
    >>> N = ReferenceFrame('N')
    >>> P = Point('P')
    >>> P.set_vel(N, u * N.x)
    >>> vel_vecs = [P.vel(N)]
    >>> gen_speeds = [u]
    >>> partial_velocity(vel_vecs, gen_speeds, N)
    [[N.x]]

    """

    if not iterable(vel_vecs):
        raise TypeError('Velocity vectors must be contained in an iterable.')

    if not iterable(gen_speeds):
        raise TypeError('Generalized speeds must be contained in an iterable')

    vec_partials = []
    for vec in vel_vecs:
        partials = []
        for speed in gen_speeds:
            partials.append(vec.diff(speed, frame, var_in_dcm=False))
        vec_partials.append(partials)

    return vec_partials

In [14]:
from sympy import expand

def partial_velocity_me(vel_vecs, gen_speeds, frame):
    
    if not iterable(vel_vecs):
        raise TypeError('Velocity vectors must be contained in an iterable.')

    if not iterable(gen_speeds):
        raise TypeError('Generalized speeds must be contained in an iterable')

    
    vec_partials = []
    repl_dicts = [{key: 0 for key in gen_speeds if key!=speed} for speed in gen_speeds]
    for vel in vel_vecs:
        partials = []
        for idx, speed in enumerate(gen_speeds):
            partial_vec = 0 * frame.x
            repl = repl_dicts[idx]
            for components, ref in vel.args:
                comp0 = components[0].xreplace(repl)
                if comp0 != 0:
                    partial_vec += ref.x * expand(comp0).coeff(speed)
                comp1 = components[1].xreplace(repl)
                if comp1 != 0:
                    partial_vec += ref.y * expand(comp1).coeff(speed)
                comp2 = components[2].xreplace(repl)
                if comp2 != 0:
                    partial_vec += ref.z * expand(comp2).coeff(speed)
            partials.append(partial_vec)
        vec_partials.append(partials)
    return vec_partials
    


In [15]:
from sympy import linear_eq_to_matrix
from sympy.physics.mechanics import Vector

def partial_velocity_timome(vel_vecs, gen_speeds, frame):
   
    if not iterable(vel_vecs):
        raise TypeError('Velocity vectors must be contained in an iterable.')

    if not iterable(gen_speeds):
        raise TypeError('Generalized speeds must be contained in an iterable')

    vec_partials = []
    gen_speeds = list(gen_speeds)
    for vel in vel_vecs:
        partials = [Vector(0) for _ in gen_speeds]
        for components, ref in vel.args:
            mat, _ = linear_eq_to_matrix(components, gen_speeds)
            for i in range(len(gen_speeds)):
                for dim, direction in enumerate(ref):
                    if mat[dim, i] != 0:
                        partials[i] += direction * mat[dim, i]
                    
        vec_partials.append(partials)

    return vec_partials

# Benchmarks

In [16]:
##### BENCHMARK 1 #####

# Measuring execution time and correctness on n_link_pendulum_on_cart example
%timeit -n 1 partialvel = partial_velocity(velocities, speeds, frame)
%timeit -n 10 partialvel_me = partial_velocity_me(velocities, speeds, frame)
%timeit -n 10 partialvel_timo =  partial_velocity_timome(velocities, speeds, frame)

partialvel = partial_velocity(velocities, speeds, frame)
partialvel_me = partial_velocity_me(velocities, speeds, frame)
partialvel_timome =  partial_velocity_timome(velocities, speeds, frame)

try:
    assert partialvel_me == partialvel
    assert partialvel_timome == partialvel
    print("No differences found between the lists.")
    
except AssertionError:
    print("Differences found between the lists.")


3.56 s ± 43.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
295 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
213 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
No differences found between the lists.


In [17]:
##### BENCHMARK 2 #####

# Measuring execution time and correctness on a test system from sympy repo
frame, velocities, speeds, map = test_issue_24887()

%timeit -n 1 partialvel = partial_velocity(velocities, speeds, frame)
%timeit -n 10 partialvel_me = partial_velocity_me(velocities, speeds, frame)
%timeit -n 10 partialvel_timo =  partial_velocity_timome(velocities, speeds, frame)

partialvel = partial_velocity(velocities, speeds, frame)
partialvel_me = partial_velocity_me(velocities, speeds, frame)
partialvel_timome =  partial_velocity_timome(velocities, speeds, frame)

try:
    assert partialvel_me == partialvel
    assert partialvel_timome == partialvel
    print("No differences found between the lists.")
    
except AssertionError:
    print("Differences found between the lists.")


43.1 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.93 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.31 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
No differences found between the lists.
